# 엑셀 파일 말고 사이트별 category id CSV 파일

In [ ]:
import pandas as pd
df = pd.read_csv('./NS_main_mid_sub1_cat.csv')
print(len(df))
df.head()

# selenium 크롤링 -> 사용안함

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

ua = UserAgent(verify_ssl=False) # inspried by Nara's code
fake_ua = ua.random
headers = { 'user-agent' : fake_ua }

url = 'https://search.shopping.naver.com/search/category/{}'.format('50003086')
res = requests.get(url, headers=headers)

soup = BeautifulSoup(res.text, 'html.parser')
# soup.select('.style_content__xWg5l .noResult_no_result__6kgzY')

mid_cat_list = soup.select('.filter_finder_list__4PE1C .filter_text_over__iesoO')
mid_cat_list

# mid_cat_list = [mid_cat.text for mid_cat in mid_cat_list] # 

In [ ]:
# 눌렀는데 없는 거면 pass
# 1. 눌렀는데 있는 건데 동일하면 pass
# 2. 동일하지 않으면 긁어옴
# 3. 여기서 만약에 들어갔는데 리스트로로 또 뽑았는데 없으면 => 검색결과가 없습니다
#        있으면 검색결과가 있는 것이라고 확인해주는 건 있으면 좋을 것 같음. 

In [ ]:
# why this process?
# => 하려고하는 메인 카테고리를 정하고 그에 따른 카테고리 이름, 아이디를 추출하는 데이터 프레임을 뽑기 위해
# => 스마트 스토어 접근이 어려웠다면? 
import re
from bs4 import BeautifulSoup
import time
from collections import defaultdict
from selenium import webdriver # webdriver를 이용해 해당 브라우저 열기
from selenium.webdriver import ActionChains # 일련의 작업들을 연속적으로 진행할 수 있도록 도와줌
from selenium.webdriver.common.keys import Keys # 키보드 입력을 할 수 있게 하기 위해
from selenium.webdriver.common.by import By # html요소 탐색을 할 수 있게 하기 위해
from selenium.webdriver.support.ui import WebDriverWait # 브라우저 응답을 기다릴 수 있게 하기 위해
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC # html 요소의 상태를 체크할 수 있게 하기 위해
import pandas as pd

# -- 처음 데이터 프레임 -- 
df = pd.read_csv('./NS_main_mid_sub1_cat.csv') 

def make_dict_by_mainCat(mainCat: str) -> dict:
    """
    mainCat별 midCat과 sub1Cat 리스트로 나누어주는 
    딕션어리를 만들어주는 함수입니다. 
    """
    main_mid_sub1_dict = {}
    
    # -- mainCat 기준으로 데이터 프레임 재생성 -- 
    df_by_main = df[df['mainCat_name'] == mainCat]
    
    # -- 데이터 프레임에서 unique한 midCat 구하기 --
    midCat_list = list(df_by_main['midCat_name'].unique())
    
    # -- midCat에 따라 sub1Cat 뽑아와서 dictionary 형태로 만들기 --
    mid_sub1_dict = defaultdict(list)
    for midCat in midCat_list:
        sub1_list = list(df_by_main[df_by_main['midCat_name'] == midCat]['sub1Cat_name'].unique()) # midCat에 따른 unique한 sub1 리스트
        mid_sub1_dict[midCat] = sub1_list
        
    # -- mid_sub1_dict와 mainCat합쳐서 새로운 딕션어리 생성 --
    main_mid_sub1_dict[mainCat] = dict(mid_sub1_dict)
    
    return main_mid_sub1_dict

def bring_sub2Cat_id_from_url():
    """
    url에서 sub2Cat_id만 가져오는 함수
    """
    current_url = dr.current_url # 현재 주소에서 가져옴
    cat_id = re.findall(r'\/\w+\?', current_url)[0].lstrip('/').rstrip('?') 
    return cat_id

def click_by_cat(cat):
    """
    category별로 클릭해주는 함수
    """
    el = dr.find_element_by_link_text(cat) 
    el.click()
    ### 여기도 clickable 하게 만드는 코드 ###
    return 

def bring_sub2Cat_Series(mainCat_dict: dict): # 대분류 딕션어리 하나 받았다고 가정하고
    
    sub2Cat_list = list()       #### 여기 바뀌어야되 ####
    for mainCat, midCat_dict in mainCat_dict.items(): # mainCat & midCat list
        for midCat, sub1Cat_list in midCat_dict.items(): # midCat & sub1Cat list
            for sub1Cat in sub1Cat_list: # sub1Cat
                
                # -- 다음 sub2Cat 리스트 뽑아내기 --
                url = 'https://search.shopping.naver.com/search/category/{}'.format(sub1Cat) # 처음부터 sub1Cat따라 url 들어오기
                dr.get(url)
                
                # -- class 부분이 클릭이 가능해질 때까지 wait --
                wait = WebDriverWait(driver, 8)
                element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'filter_finder_list__4PE1C')))
                
                # -- beatifulsoup으로 sub2에 해당하는 리스트 가져오기 -- 
                soup = BeautifulSoup(dr.page_source, 'html.parser')
                sub2_list = soup.select('.filter_finder_list__4PE1C .filter_text_over__iesoO')
                sub2_list = [sub2.text for sub2 in sub2_list]
                
                if (sub2_list == sub1Cat_list) | (sub2_list == []):
                    sub2Cat_list.append([sub1Cat, ' ', ' ']) # sub2가 없는 경우 빈칸과 '_' 조인
                    
                else:    
                    for sub2Cat_name in sub2_list:
                        # -- sub2Cat 클릭 --
                        click_by_cat(sub2Cat)
                        # -- 잠시 대기 --
                        dr.implicitly_wait(3)
                        # -- url에서 카테고리 아이디 가져오기 --
                        sub2Cat_id = bring_sub2Cat_id_from_url()
                        sub2Cat_list.append([sub1Cat, sub2Cat_id, sub2Cat_name]) # sub2 있는 경우 '_' 조인
                        # -- 다시 뒤로 돌아가기 -- 
                        click_by_cat(sub1Cat)
                        # -- 잠기 대기 --
                        dr.implicitly_wait(3)
                
                dr.close() # 다음 카테고리로 넘어가기 위해 창 닫아주기
                time.sleep(0.07)
    sub2Cat_df = pd.DataFrame(sub2Cat_list, columns=['sub1Cat_name', 'sub2Cat_id', 'sub2Cat_name'])
    return sub2Cat_df  # 기존 데이터 프레임에 merge시키기 위해 시리즈 데이터로 변경 
    
def merge_mainCat_sub2Cat_df(mainCat):
    mainCat_dict = make_dict_by_mainCat(mainCat)
    sub2Cat_df = bring_sub2Cat_Series(mainCat_dict)
    df_by_main = df[df['mainCat_name'] == mainCat]
    final_df = pd.merge(df_by_main, sub2Cat_df, left_on='sub1Cat_name', right_on='sub1Cat_name', how='left')
    
    return final_df # sub2Cat_name, sub2Cat_id까지 합쳐진 mainCat 별 데이터 프레임


# 1. series 데이터 '_'로 분리시켜서 데이터 프레임으로 변경뒤
# 2. 기존 데이터 프레임에 left merge 시켜주기 



In [ ]:
# 심지어 없는 부분에 대해서도 알아서 Nan 처리를 해줌 => 거의 다 왔다 조금만..!
pd.merge(tmp_df, tmp_df2, left_on='sub1Cat_name', right_on = 0, how='left')

# Open API 크롤링 -> 안씀

In [ ]:
import requests
import urllib

query = "남성 모자"
query = urllib.parse.quote(query)

display = "10"

url = "https://openapi.naver.com/v1/search/shop?query=" + query + "&display=" + display

request = urllib.request.Request(url)
request.add_header('X-Naver-Client-Id', '5oXnmo8N6zm_ipiF9EEr')
request.add_header('X-Naver-Client-Secret', '3y6lUnczAb')

response = urllib.request.urlopen(request)
print(response.read().decode('utf-8'))

# requests, json 크롤링 -> 최종

In [ ]:
# !pip install pycryptodome

***catId를 넣어 크롤링 해오는 함수***

In [ ]:
import requests
from fake_useragent import UserAgent
import json
import time
import pandas as pd
from sbth import get_sbth

def get_ns_prd_list(cat_id: str, page_num: int) ->list:
    sbth = get_sbth()
    ua = UserAgent(verify_ssl=False) 
    fake_ua = ua.random
    headers = {
        'user-agent' : fake_ua,
        'cache-control' : 'no-cache, no-store, must-revalidate',
        # -- sbth 없으면 정보 안가져옴 --
        # -- update 필요할 수 있음 => 날짜 정보가 들어가 만료가 될 수 있음 -- 
        'sbth' : sbth,
        # -- referer 없으면 정보 안가져옴 --
        'referer' : 'https://search.shopping.naver.com/search/category',    
    }

    params = (
        ('sort','rel'),
        ('pagingIndex',page_num),
        ('pagingSize','40'),
        ('viewType','list'),
        # -- 여기에 해당하는 catId는 엑셀파일에 있는 catId --
        ('catId', cat_id),
    )


    url = 'https://search.shopping.naver.com/api/search/category'
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        results = res.json()
        results = results['shoppingResult']['products'] # products 키 값안에 상품 정보 들어있음. 
        
        prd_list = list()
        for result in results:
            prd_id = result['id']
            prd_name = result['productName']
            cat1_id = result['category1Id']
            cat1_name = result['category1Name']
            cat2_id = result['category2Id']
            cat2_name = result['category2Name']
            cat3_id = result['category3Id']
            cat3_name = result['category3Name']
            # -- cat 4 name -- 
            
            prd_image_url = result['imageUrl']
            prd_low_price = result['lowPrice']
            prd_list.append([prd_id, prd_name, cat1_id, cat1_name, cat2_id, 
                             cat2_name, cat3_id, cat3_name, prd_image_url, prd_low_price]) 
    else:
        prd_list.append([None, None, None, None, None,
                         None, None, None, None, None]) # 변수들 None 값으로 처리 
        
    return prd_list



def get_prd_info(cat_id: str, page_count = 250):
    """
    cat_id -> 크롤링하고자 하는 category id
    page_count -> 크롤링하고자 하는 페이지 수 (250 pages by default)
    """
    start_time = time.time()
    prd_list = list()
    for page_num in range(1, page_count+1): # 250 페이지 가져오기 
        prd_list += get_ns_prd_list(cat_id, page_num)
        print('page %d done'%page_num)
        
        if page_num % 50 == 0:
            print('-- paused for 5 sec in case not to be blocked --')
            time.sleep(5)
            
    print('-- 데이터 개수 :  %d --'%(len(prd_list)))
    
    # -- prd_list에서 데이터 프레임 만들기 --
    df = pd.DataFrame(prd_list, columns = ['prd_id', 'prd_name', 'cat1_id', 'cat1_name', 'cat2_id', 
                                           'cat2_name', 'cat3_id', 'cat3_name', 'prd_image_url', 
                                           'prd_low_price'])
    end_time = time.time()
    print(end_time - start_time)
    return df


get_prd_info('50000819', 2)



***catID url용 베이스 코드***

In [ ]:
import requests
from fake_useragent import UserAgent
import json
import time
import pandas as pd
from sbth import get_sbth

sbth = get_sbth()
ua = UserAgent(verify_ssl=False) 
fake_ua = ua.random
headers = {
    'user-agent' : fake_ua,
    'cache-control' : 'no-cache, no-store, must-revalidate',
    # -- sbth 없으면 정보 안가져옴 --
    # -- update 필요할 수 있음 => 날짜 정보가 들어가 만료가 될 수 있음 -- 
    'sbth' : sbth,
    # -- referer 없으면 정보 안가져옴 --
    'referer' : 'https://search.shopping.naver.com/search/category',    
}

params = (
    ('sort','rel'),
    ('pagingIndex','2'),
    ('pagingSize','40'),
    ('viewType','list'),
    # -- 여기에 해당하는 catId는 엑셀파일에 있는 catId --
#     ('catId', cat_id),
)


# url = 'https://search.shopping.naver.com/api/search/category'
# res = requests.get(url, headers=headers, params=params)



# url = 'https://search.shopping.naver.com/api/search/category/100005522'
# url = 'https://search.shopping.naver.com/api/search/category/100005522?catId=50000832'
url = 'https://search.shopping.naver.com/api/search/category/100000386?catId=50000425%2050000435%2050001487%2050001488'
res = requests.get(url, headers=headers, params=params)
data = res.json()
data['shoppingResult']




***검색 쿼리용 베이스 코드***

In [ ]:
import requests
from fake_useragent import UserAgent
import json
import time
import pandas as pd
from sbth import get_sbth


sbth = get_sbth() # sbth 가져오기
ua = UserAgent(verify_ssl=False) # UserAgent 생성
fake_ua = ua.random # UserAgent 생성

headers = {
    'user-agent' : fake_ua,
    'cache-control' : 'no-cache, no-store, must-revalidate',
    # -- sbth 없으면 정보 안가져옴 --
    # -- 자동 sbth 생성 코드 적용 -- 
    'sbth' : sbth,
    # -- referer 없으면 정보 안가져옴 --
    'referer' : 'https://search.shopping.naver.com/search/all'
}

params = (
    ('sort','rel'),
#     ('origQuery', query),
#     ('query', query),
    ('productSet', 'total'), # 이게 뭐지?
    ('pagingIndex', '2'),
    ('pagingSize','40'),
    ('viewType','list'),
) 

url = 'https://search.shopping.naver.com/api/search/all?query=여성항공점퍼'
res = requests.get(url, headers=headers, params=params)
data = res.json()
data['shoppingResult']['products']
    


***크롤링에 사용될 수 있도록 url 전처리 해주는 함수***

In [ ]:
import re
import pandas as pd

def get_final_url_df():
    """
    카테고리 매칭 작업했던 파일을 가져오고
    최종 url에서 'api'단어를 붙이기 위해 준비해주는 함수. 
    """
    df = pd.read_csv('./ns_카테고리 매칭 - 최종 카테고리.csv', header=1)
    df['temp URL'] = df['temp URL'].fillna(df['최종 URL']) # 비어있는 temp URL 부분을 최종 URL로 채워줍니다. 
    return df 

def add_api_into_url(url: str) -> str:
    """
    url 안에 'api'단어를 붙여주는 함수.
    'api'단어가 있어야 크롤링 가능. 
    """
    new_url = None
    try:
        new_url = re.sub('(.com\/search)', '.com/api/search', url)
    except:
        new_url = 'no_url'
    return new_url

def return_url_list():
    """
    크롤링이 가능한 url을 반환해주는 함수.
    """
    df = get_final_url_df()
    df['url_for_crawling'] = df['temp URL'].map(lambda x : add_api_into_url(x))
    url_list = list(df['url_for_crawling'])
    url_list = [url for url in url_list if url != 'no_url']
    
    # -- 리스트 txt 파일로 만들어주기 -- 
    with open('ns_urls.txt', 'w+') as file:
        file.write(','.join(url_list))
        
    return 

from ns_make_urls_for_crawling import return_url_list
return_url_list()

# list로 다시 불러오기
with open('./ns_urls.txt', 'r') as f:
    naver_urls = f.read()
naver_url_list = naver_urls.split(',')

print(len(naver_url_list))

In [4]:
from ns_make_urls_for_crawling import return_url_list
return_url_list()

***크롤링 최종 함수***

In [ ]:
import requests
from fake_useragent import UserAgent
import json
import time
import pandas as pd
from sbth import get_sbth

def get_ns_prd_list(url: str, page_num: int) ->list:
    
    sbth = get_sbth()
    ua = UserAgent(verify_ssl=False) 
    fake_ua = ua.random
    headers = {
        'user-agent' : fake_ua,
        'cache-control' : 'no-cache, no-store, must-revalidate',
        # -- sbth 없으면 정보 안가져옴 --
        'sbth' : sbth,
        # -- referer 없으면 정보 안가져옴 --
        'referer' : 'https://search.shopping.naver.com/search/category',    
    }

    params = (
        ('sort','rel'),
        ('pagingIndex', page_num),
#         ('productSet', 'total'), # 전체 탭 보여주는 파라미터 (가격비교, 네이버페이 등으로 변환 가능)
        ('pagingSize','40'),
        ('viewType','list'),
    )
        
    prd_list = list()
    res = requests.get(url, headers=headers, params=params)
    if (res.status_code == 200):
        results = res.json()
        results = results['shoppingResult']['products'] # products 키 값안에 상품 정보 들어있음. 
        
        for result in results:
            prd_id = result['id']
            prd_name = result['productName']
            cat1_id = result['category1Id']
            cat1_name = result['category1Name']
            cat2_id = result['category2Id']
            cat2_name = result['category2Name']
            cat3_id = result['category3Id']
            cat3_name = result['category3Name']
            cat4_id = result['category4Id']
            cat4_name = result['category4Name']
            
            prd_image_url = result['imageUrl']
            prd_low_price = result['lowPrice']
            prd_list.append([prd_id, prd_name, cat1_id, cat1_name, cat2_id, 
                             cat2_name, cat3_id, cat3_name, cat4_id, cat4_name, 
                             prd_image_url, prd_low_price]) 
    else:
        pass
        
    return prd_list



def get_prd_info(url: str, page_count = 250):
    """
    cat_id -> 크롤링하고자 하는 category id
    page_count -> 크롤링하고자 하는 페이지 수 (250 pages by default)
    """
    prd_list = list()
    for page_num in range(1, page_count+1): # 250 페이지 가져오기 
        prd_list += get_ns_prd_list(url, page_num)
        print('page %d done'%page_num)
        
        if page_num % 50 == 0:
            print('-- paused for 5 sec in case not to be blocked --')
            time.sleep(5)
            
    print('-- 데이터 개수 :  %d --'%(len(prd_list)))
    
    # -- prd_list에서 데이터 프레임 만들기 --
    df = pd.DataFrame(prd_list, columns = ['prd_id', 'prd_name', 'cat1_id', 'cat1_name', 'cat2_id', 
                                           'cat2_name', 'cat3_id', 'cat3_name', 'cat4_id', 'cat4_name',
                                           'prd_image_url', 'prd_low_price'])
    return df


# url = 'https://search.shopping.naver.com/api/search/category/100005542?catId=50000841'
url = 'https://search.shopping.naver.com/api/search/all?query=여성항공점퍼'
get_prd_info(url, 3)